<a href="https://colab.research.google.com/github/MsMore/Vaani/blob/main/finetunevgg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
from keras import utils, callbacks
from keras.models import Sequential
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split, StratifiedKFold
#from keras.preprocessing.image import load_img, img_to_array
from skimage.transform import resize
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_directory = "/content/drive/MyDrive/Image/Train"
test_directory = "/content/drive/MyDrive/Image/Validation"

In [ ]:
from keras.applications import VGG16
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(128,128, 3))

58892288/58889256 [==============================] - 1s 0us/step


In [ ]:
# Freeze four convolution blocks
for layer in vgg_model.layers[:15]:
    layer.trainable = False
# Make sure you have frozen the correct layers
for i, layer in enumerate(vgg_model.layers):
    print(i, layer.name, layer.trainable)

0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 True
16 block5_conv2 True
17 block5_conv3 True
18 block5_pool True


In [ ]:
x = vgg_model.output
x = Flatten()(x) # Flatten dimensions to for use in FC layers
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x) # Dropout layer to reduce overfitting
x = Dense(256, activation='relu')(x)
x = Dense(28, activation='softmax')(x) # Softmax for multiclass
transfer_model = Model(inputs=vgg_model.input, outputs=x)

In [ ]:
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=8, verbose=1, mode='max', min_lr=5e-5)
checkpoint = ModelCheckpoint('vgg16_finetune.h15', monitor= 'val_accuracy', mode= 'max', save_best_only = True, verbose= 1)

In [ ]:
gen = ImageDataGenerator(rescale=1./255, validation_split=0.1) 
train = gen.flow_from_directory(train_directory, target_size=(128, 128), subset="training")
val = gen.flow_from_directory(test_directory, target_size=(128, 128), subset="validation")

Found 60480 images belonging to 28 classes.
Found 1680 images belonging to 28 classes.


In [ ]:
from tensorflow.keras import layers, models, Model, optimizers
learning_rate= 5e-5
transfer_model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=learning_rate), metrics=["accuracy"])
history = transfer_model.fit(train, batch_size = 1, epochs=50, validation_data=val, callbacks=[lr_reduce,checkpoint])

Epoch 1/50
1890/1890 [==============================] - 23826s 13s/step - loss: 0.8324 - accuracy: 0.7616 - val_loss: 0.2558 - val_accuracy: 0.9155

Epoch 00001: val_accuracy improved from -inf to 0.91548, saving model to vgg16_finetune.h15
INFO:tensorflow:Assets written to: vgg16_finetune.h15/assets
Epoch 2/50
1890/1890 [==============================] - 7402s 4s/step - loss: 0.0198 - accuracy: 0.9945 - val_loss: 0.3549 - val_accuracy: 0.9333

Epoch 00002: val_accuracy improved from 0.91548 to 0.93333, saving model to vgg16_finetune.h15
INFO:tensorflow:Assets written to: vgg16_finetune.h15/assets
Epoch 3/50
1890/1890 [==============================] - 7404s 4s/step - loss: 0.0102 - accuracy: 0.9975 - val_loss: 0.5121 - val_accuracy: 0.9304

Epoch 00003: val_accuracy did not improve from 0.93333
Epoch 4/50
1890/1890 [==============================] - 7323s 4s/step - loss: 0.0090 - accuracy: 0.9977 - val_loss: 0.3157 - val_accuracy: 0.9411

Epoch 00004: val_accuracy improved from 0.9333

KeyboardInterrupt: ignored

In [ ]:
for layer in vgg_model.layers[:15]:
  layer.trainable = False
x = vgg_model.output
x = Flatten()(x) # Flatten dimensions to for use in FC layers
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x) # Dropout layer to reduce overfitting
x = Dense(256, activation='relu')(x)
x = Dense(28, activation='softmax')(x) # Softmax for multiclass
transfer_model = Model(inputs=vgg_model.input, outputs=x)
for i, layer in enumerate(transfer_model.layers):
  print(i, layer.name, layer.trainable)

0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 True
16 block5_conv2 True
17 block5_conv3 True
18 block5_pool True
19 flatten_1 True
20 dense_3 True
21 dropout_1 True
22 dense_4 True
23 dense_5 True


In [ ]:
#Augment images
train_datagen = ImageDataGenerator(zoom_range=0.2, rotation_range=30, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2)
#Fit augmentation to training images
train_generator = train_datagen.flow(train,batch_size=1)
#Compile model
transfer_model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])
#Fit model
history = transfer_model.fit_generator(train_generator, validation_data=val, epochs=8, shuffle=True, callbacks=[lr_reduce],verbose=1)

NameError: ignored